In [32]:
from bs4 import BeautifulSoup
import requests
from datetime import datetime
import pandas as pd
from datetime import timedelta

page = requests.get("http://www.90minut.pl/liga/1/liga12330.html")
soup = BeautifulSoup(page.content, features="html.parser")
scores = soup.findAll("a", {"class": "main"})
dates = soup.findAll("td", valign="top", align="left",width="190")
soup = soup.findAll("td", valign="top", width="180")
months = {
    "stycznia": "January",
    "lutego": "February",
    "marca": "March",
    "kwietnia": "April",
    "maja": "May",
    "czerwca": "June",
    "lipca": "July",
    "sierpnia": "August",
    "września": "September",
    "października": "October",
    "listopada": "November",
    "grudnia": "December"
}

In [3]:
def delete_comma(text):
    if ',' in text:
        return text.split(',')[0]
    else:
        return text

In [4]:
def pl_months_to_eng(data_str):


    day, month, year = data_str.split()


    month = months[month]


    data_str_en = f"{day} {month} {year}"


    data = datetime.strptime(data_str_en, "%d %B %Y")
    return data

In [5]:
def add_points(team,points,tabela):
       tabela[tabela(tabela["Team_name"].equals(team)).index,1]=+points

In [6]:
EXTRA_POINTS_WIN=0.2
EXTRA_POINTS_DRAW=0
POINTS_WIN=3
POINTS_DRAW=1
POINTS_LOST=0.0

In [7]:
team = []
score = []
date=[]
new_date=[]
for i in soup:
    team.append(i.getText())
for i in scores:
    score.append(i.getText())
for i in dates:
    date.append(i.getText())

score=score[:len(score)-2]
score=score[-307:]
score=score[:len(score)]
score.remove('(wo) ')
new_date=list(map(delete_comma,date))

home=[]
away=[]
for i in range(0,len(team)):
    if i%2==0:
        home.append(team[i])
    else:
        away.append(team[i])   
for i in range(0,len(new_date)):
    if "lipca" in new_date[i] or "sierpnia" in new_date[i] or "września" in new_date[i] or "października" in new_date[i] or "listopada" in new_date[i]:
        new_date[i]+=" 2021"
    else:
        new_date[i]+= " 2022"
        

    

In [8]:
t_date=list(map(pl_months_to_eng,new_date))


In [9]:
home_goal=[]
away_goal=[]
for i in score:
    home_goal.append(i[0])
    away_goal.append(i[2])



In [10]:
dataMatrix={'Home_team':home,
            'Home_score':list(map(int,home_goal)),
            'Away_score':list(map(int,away_goal)),
            'Away_team':away,
            'date':t_date}
out_data=pd.DataFrame(dataMatrix)


In [11]:
points_home=[]
points_away=[]
for i in range(0,len(out_data.iloc[:,0])):
    if (out_data.iloc[i,1]>out_data.iloc[i,2]):
        points_home.append(POINTS_WIN)
        points_away.append(POINTS_LOST)
    elif (out_data.iloc[i,1]<out_data.iloc[i,2]):
        points_home.append(POINTS_LOST)
        points_away.append(POINTS_WIN)
    else:
        points_home.append(POINTS_DRAW)
        points_away.append(POINTS_DRAW)
out_data["Points_home"]=points_home
out_data["Points_away"]=points_away
print(out_data)          

                     Home_team  Home_score  Away_score           Away_team  \
0              Górnik Zabrze             0           2          Cracovia     
1      Jagiellonia Białystok             2           0     Piast Gliwice     
2              Korona Kielce             1           1    Legia Warszawa     
3                Lech Poznań             0           2       Stal Mielec     
4             Pogoń Szczecin             2           1       Widzew Łódź     
..                         ...         ...         ...                 ...   
301            Piast Gliwice             3           0     Lechia Gdańsk     
302           Pogoń Szczecin             4           0    Radomiak Radom     
303        Raków Częstochowa             1           1    Zagłębie Lubin     
304              Stal Mielec             1           0      Warta Poznań     
305              Widzew Łódź             0           3     Korona Kielce     

          date  Points_home  Points_away  
0   2021-07-18      

In [12]:
team=out_data["Home_team"].unique()
points=[0]*len(team)
Table={"Team_name":team,
      "Points":points}
Table=pd.DataFrame(Table)



In [13]:

for i in range(0,len(out_data["Home_team"])):    
    p=out_data.iloc[i]["Points_home"]
    t=out_data.iloc[i]["Home_team"]
    for j in range(0,len(Table['Team_name'])):
        if Table.iloc[j]['Team_name'] == t:
            Table.iat[j,1]=Table.iat[j,1]+p       
    p=out_data.iloc[i]["Points_away"]
    t=out_data.iloc[i]["Away_team"]
    for j in range(0,len(Table['Team_name'])):
        if Table.iloc[j]['Team_name'] == t:
            Table.iat[j,1]=Table.iat[j,1]+p      
    



In [14]:
out_data=out_data.sort_values(by=['date'], ascending=True)
print(out_data)

                     Home_team  Home_score  Away_score           Away_team  \
6          Raków Częstochowa             1           0      Warta Poznań     
8             Zagłębie Lubin             0           0     Śląsk Wrocław     
1      Jagiellonia Białystok             2           0     Piast Gliwice     
2              Korona Kielce             1           1    Legia Warszawa     
3                Lech Poznań             0           2       Stal Mielec     
..                         ...         ...         ...                 ...   
300            Miedź Legnica             0           0     Górnik Zabrze     
301            Piast Gliwice             3           0     Lechia Gdańsk     
302           Pogoń Szczecin             4           0    Radomiak Radom     
303        Raków Częstochowa             1           1    Zagłębie Lubin     
305              Widzew Łódź             0           3     Korona Kielce     

          date  Points_home  Points_away  
6   2021-07-15      

In [15]:
points_home=[]
points_away=[]
for i in range(0,len(out_data.iloc[:,0])):
    if (out_data.iloc[i,1]>out_data.iloc[i,2]):
        points_home.append(EXTRA_POINTS_WIN)
        points_away.append(-EXTRA_POINTS_WIN)
    elif (out_data.iloc[i,1]<out_data.iloc[i,2]):
        points_home.append(-EXTRA_POINTS_WIN)
        points_away.append(EXTRA_POINTS_WIN)
    else:
        points_home.append(EXTRA_POINTS_DRAW)
        points_away.append(EXTRA_POINTS_DRAW)
out_data["Home_team_temp_points"]=points_home
out_data["Away_team_temp_points"]=points_away

In [47]:
tmpTeam=(out_data[out_data["Away_team"].str.contains(team[0]) | out_data["Home_team"].str.contains(team[0])])

dateDiff=[]
for i in range(1,len(tmpTeam.iloc[:,0])):
    dateDiff.append(tmpTeam.iloc[i][4]-tmpTeam.iloc[i-1][4])
dateDiff.insert(0,0)

tmpTeam["Data_difference"]=dateDiff


tmpTeamPoints=[]
for i in range (0,len(tmpTeam.iloc[:,0])):
    if(tmpTeam.iloc[i][0]==team[0]):
        tmpTeamPoints.append(tmpTeam.iloc[i][7])
    if(tmpTeam.iloc[i][3]==team[0]):
        tmpTeamPoints.append(tmpTeam.iloc[i][8])
                



dateDiff=[i.total_seconds() for i in dateDiff if isinstance(i,timedelta)]

dateDiff.insert(0,0)
print(max(dateDiff))
addedPoints=[]
j=0
for i in range(len(tmpTeamPoints)): 
    if i<4:
        addedPoints.append(tmpTeamPoints[i])
    else:
        if (dateDiff[i]==max(dateDiff) and j==0) or (j<4 and j>0):
            print(j,dateDiff[i],i)
            j+=1
            addedPoints.append(tmpTeamPoints[i])
        else:    
            addedPoints.append(round(sum(tmpTeamPoints[i-4:i]),2))
print(addedPoints)

6307200.0
0 6307200.0 17
1 432000.0 18
2 691200.0 19
3 518400.0 20
[-0.2, 0.2, 0.2, -0.2, 0.0, 0.2, 0.0, -0.2, 0.0, 0.2, 0.0, -0.2, -0.4, -0.6, -0.2, 0.2, 0.2, -0.2, 0.0, 0.0, -0.2, -0.4, 0.0, -0.2, -0.4, 0.0, -0.4, -0.2, 0.2, 0.2, 0.6, 0.8, 0.8, 0.8]


C:\Users\admin\AppData\Local\Temp\ipykernel_17180\2697240330.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmpTeam["Data_difference"]=dateDiff
